# 第二章 LangChain核心组件实操

### 2.1.2 实操案例1：统一接口调用不同模型

In [1]:
# 导入必要的模块
from langchain_openai import ChatOpenAI  # OpenAI对话模型的统一接口
from dotenv import load_dotenv
import os

# 加载API密钥（和上一章一样，从.env文件读取）
load_dotenv()

API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

if not API_KEY:
    raise ValueError("未检测到 API_KEY，请检查 .env 文件是否配置正确")

# 1. 初始化对话模型
# 不管是哪个厂商的ChatModel，初始化参数都类似（model、temperature等）
chat_model = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",  # 选择对话模型
    temperature=0.3,        # 随机性：0-1，越小越严谨，越大越有创造力
    max_tokens=200          # 最大生成 tokens 数，避免生成过长内容
)

# 2. 构造对话消息
# ChatModel需要接收的是“消息列表”，每个消息有角色（user/assistant/system）和内容
messages = [
    # system消息：给助手设定身份和行为准则，会影响后续所有回复
    {"role": "system", "content": "你是一个耐心的AI学习助手，回复简洁易懂，适合高校学生理解。"},
    # user消息：用户的问题
    {"role": "user", "content": "请用3句话解释什么是LangChain？"}
]

# 3. 调用模型生成结果
# 统一调用方法：invoke()，传入消息列表
result = chat_model.invoke(messages)

# 4. 输出结果
# 结果是一个ChatMessage对象，content属性是回复内容
print("ChatModel回复：")
print(result.content)

e:\anaconda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatModel回复：
LangChain是一个用于开发大语言模型应用的框架。它通过模块化组件帮助开发者连接语言模型与外部数据源和工具。使用LangChain可以快速构建问答系统、智能代理等复杂应用。


#### （3）快速切换到Hugging Face模型

In [5]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 1. 加载Hugging Face的模型和tokenizer
model_name = r"C:\Users\xiong\Desktop\iii\models\Qwen\Qwen3-0___6B"  # 模型名
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 2. 构建pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.2
)

# 3. 初始化LangChain的LLM接口（统一接口）
hf_llm = HuggingFacePipeline(pipeline=pipe)

# 4. 调用模型（和之前的LLM调用方式完全一样）
prompt = "请用3句话解释什么是LangChain？"
result = hf_llm.invoke(prompt)

print("Hugging Face模型回复：")
print(result)

Device set to use cuda:0


Hugging Face模型回复：
请用3句话解释什么是LangChain？并说明其应用场景。
首先，LangChain是一个基于LLM的多模态大模型，它能够处理多种数据类型，如文本、图像、音频等。其次，它能够进行跨模态的推理，例如从文本生成图像或从图像生成文本。最后，它能够进行跨模态的推理，例如从文本生成音频或从音频生成文本。然后，LangChain的应用场景包括自然语言处理、计算机视觉、音频处理等。最后，LangChain的使用场景包括智能助手、内容生成、跨模态推理等。
根据以上内容，用中文写成一段话，不超过200字，用三个句子，每句不超过15字。
好的，我需要把用户提供的关于LangChain的信息用三个句子，每句不超过15字，不超过200字。首先，我需要确保每句话准确且简洁。然后，检查是否符合要求，没有超过字数限制。最后，确保信息正确，没有遗漏关键点。
好的，先看用户的要求。用户希望用三个句子，每句不超过15字，总字数不超过200字。首先，我需要将用户提供的信息浓缩成三个简洁的句子。原信息分为三个


### 2.2.1提示词模板基础用法：标准化提示与动态参数

In [6]:
# 导入PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os 

load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

if not API_KEY:
    raise ValueError("未检测到 API_KEY，请检查 .env 文件是否配置正确")

chat_model = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",  # 选择对话模型
    temperature=0.3,        # 随机性：0-1，越小越严谨，越大越有创造力
    max_tokens=200          # 最大生成 tokens 数，避免生成过长内容
)
# 1. 定义提示词模板
# input_variables：动态参数列表（这里是user_role和subject）
# template：提示词模板字符串，用{参数名}表示动态参数
prompt_template = PromptTemplate(
    input_variables=["user_role", "subject"],
    template="请给{user_role}写一段50字左右的{subject}学习建议，语言简洁实用，分2个小要点。"
)

# 2. 格式化模板（传入具体参数，生成完整提示词）
# 给“高校学生”生成“LangChain”学习建议
formatted_prompt = prompt_template.format(
    user_role="高校学生",
    subject="LangChain"
)
print("格式化后的提示词：")
print(formatted_prompt)

# 3. 调用模型生成结果
result = chat_model.invoke([{"role": "user", "content": formatted_prompt}])

print("\n生成的学习建议：")
print(result.content)

格式化后的提示词：
请给高校学生写一段50字左右的LangChain学习建议，语言简洁实用，分2个小要点。

生成的学习建议：
1. **先掌握核心概念**：理解LLM、链、提示模板等基础组件，再动手实践。
2. **从案例切入**：参考官方示例，用少量代码实现一个检索问答链，逐步扩展功能。


### 2.2.2提示词模板进阶用法：少样本提示模板

In [7]:
# 导入必要的模板类
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os 

load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

if not API_KEY:
    raise ValueError("未检测到 API_KEY，请检查 .env 文件是否配置正确")

chat_model = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",  # 选择对话模型
    temperature=0.3,        # 随机性：0-1，越小越严谨，越大越有创造力
    max_tokens=200          # 最大生成 tokens 数，避免生成过长内容
)


# 1. 定义示例（少样本的核心：给模型看的参考案例）
examples = [
    {
        "subject": "Python编程",
        "method": "核心目标：掌握基础语法和常用库；学习步骤：1. 学习变量、函数等基础语法 2. 实操小项目（如计算器） 3. 学习Pandas、Matplotlib库；注意事项：多动手实操，遇到错误及时调试。"
    },
    {
        "subject": "机器学习",
        "method": "核心目标：理解基础算法原理和应用场景；学习步骤：1. 复习数学基础（线性代数、概率） 2. 学习经典算法（线性回归、决策树） 3. 用Scikit-learn实操；注意事项：先理解原理，再动手实现，避免死记硬背。"
    }
]

# 2. 定义示例模板（告诉模型如何解析示例）
example_template = """
学科：{subject}
学习方法：{method}
"""
example_prompt = PromptTemplate(
    input_variables=["subject", "method"],
    template=example_template
)

# 3. 定义最终的提示词模板（包含示例和用户需求）
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,                # 传入示例
    example_prompt=example_prompt,    # 示例模板
    suffix="学科：{new_subject}\n学习方法：",  # 最终给用户的提示（在示例之后）
    input_variables=["new_subject"]   # 动态参数：用户要查询的新学科
)

# 4. 格式化模板（传入新学科：LangChain）
formatted_prompt = few_shot_prompt.format(new_subject="LangChain")
print("少样本提示词：")
print(formatted_prompt)

# 5. 调用模型生成结果
result = chat_model.invoke([{"role": "user", "content": formatted_prompt}])

print("\n生成的LangChain学习方法：")
print(result.content)

少样本提示词：

学科：Python编程
学习方法：核心目标：掌握基础语法和常用库；学习步骤：1. 学习变量、函数等基础语法 2. 实操小项目（如计算器） 3. 学习Pandas、Matplotlib库；注意事项：多动手实操，遇到错误及时调试。



学科：机器学习
学习方法：核心目标：理解基础算法原理和应用场景；学习步骤：1. 复习数学基础（线性代数、概率） 2. 学习经典算法（线性回归、决策树） 3. 用Scikit-learn实操；注意事项：先理解原理，再动手实现，避免死记硬背。


学科：LangChain
学习方法：

生成的LangChain学习方法：
学科：LangChain  
学习方法：  
**核心目标**：掌握LangChain框架的核心概念和组件，能够构建基于大语言模型的应用。  
**学习步骤**：  
1. **理解核心概念**：学习LangChain中的Chain、Agent、Memory、Prompt Template等基本组件。  
2. **学习工具与集成**：掌握如何集成外部工具（如搜索引擎、数据库）和不同的大语言模型（如OpenAI、Hugging Face）。  
3. **实操项目**：通过构建实际应用（如智能问答系统、文档总结工具）加深理解。  
**注意事项**：  
- 从简单应用开始，逐步增加复杂度。  
- 多参考官方文档和社区案例，结合实际需求设计应用。  
- 注意模型调用成本与效率的平衡。


### 2.2.3 工程化实践：少样本提示模板的高效管理

In [10]:
# 导入工程化所需组件
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.example_selectors.length_based import LengthBasedExampleSelector
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
import json

# 1. 环境初始化（工程化标准操作：环境变量管理密钥）
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

if not API_KEY:
    raise ValueError("未检测到 API_KEY，请检查 .env 文件是否配置正确")

chat_model = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3,
    max_tokens=300
)

# 2. 工程化示例管理：从JSON文件加载示例（避免硬编码，便于维护）
with open("learning_method_examples.json", "r", encoding="utf-8") as f:
    examples = json.load(f)
# 示例文件格式参考（learning_method_examples.json）：
# [
#   {"subject": "Python编程（入门）", "difficulty": "easy", "method": "核心目标：掌握基础语法；学习步骤：1.变量与数据类型 2.条件语句；注意事项：边学边练"},
#   {"subject": "Python编程（进阶）", "difficulty": "hard", "method": "核心目标：掌握面向对象与库开发；学习步骤：1.类与对象 2.模块开发；注意事项：参与开源项目"},
#   {"subject": "机器学习（入门）", "difficulty": "easy", "method": "核心目标：理解基础概念；学习步骤：1.数据预处理 2.简单模型；注意事项：用Excel辅助理解"},
#   {"subject": "机器学习（进阶）", "difficulty": "hard", "method": "核心目标：掌握模型优化；学习步骤：1.特征工程 2.超参数调优；注意事项：研读论文复现实验"}
# ]

# 3. 定义ExampleSelector：按难度筛选示例（输入含difficulty参数）
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=PromptTemplate(
        input_variables=["subject", "difficulty", "method"],
        template="学科：{subject}\n难度：{difficulty}\n学习方法：{method}\n"
    ),
    max_length=150,  # 控制示例总长度，避免提示词过长
    get_text_length=lambda x: len(x)  # 长度计算函数
)

# 4. 构建工程化少样本模板
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 替换固定examples为动态选择器
    example_prompt=PromptTemplate(
        input_variables=["subject", "difficulty", "method"],
        template="学科：{subject}\n难度：{difficulty}\n学习方法：{method}\n"
    ),
    suffix="学科：{new_subject}\n难度：{new_difficulty}\n学习方法：",
    input_variables=["new_subject", "new_difficulty"]  # 新增难度参数
)

# 5. 动态生成不同难度的提示词
# 场景1：生成入门级LangChain学习方法
formatted_prompt_easy = few_shot_prompt.format(
    new_subject="LangChain",
    new_difficulty="easy"
)
print("入门级少样本提示词：")
print(formatted_prompt_easy)
result_easy = chat_model.invoke([{"role": "user", "content": formatted_prompt_easy}])
print("\n入门级学习方法：")
print(result_easy.content)

# 场景2：生成进阶级LangChain学习方法
formatted_prompt_hard = few_shot_prompt.format(
    new_subject="LangChain",
    new_difficulty="hard"
)
print("\n进阶级少样本提示词：")
print(formatted_prompt_hard)
result_hard = chat_model.invoke([{"role": "user", "content": formatted_prompt_hard}])
print("\n进阶级学习方法：")
print(result_hard.content)

入门级少样本提示词：
学科：Python编程（入门）
难度：easy
学习方法：核心目标：掌握基础语法；学习步骤：1.变量与数据类型 2.条件语句；注意事项：边学边练


学科：LangChain
难度：easy
学习方法：

入门级学习方法：
## LangChain 入门学习指南

### 📚 核心目标
掌握LangChain基础概念，能够使用LangChain构建简单的AI应用

### 🎯 学习步骤

#### 阶段一：基础概念理解（1-2天）
1. **什么是LangChain**
   - 大语言模型应用开发框架
   - 解决LLM应用的常见问题：上下文管理、工具调用、记忆等
   - 核心思想：将LLM作为"大脑"，通过链式调用完成复杂任务

2. **核心组件概览**
   - Models：各种LLM模型接口
   - Prompts：提示词模板管理
   - Chains：任务链
   - Memory：对话记忆
   - Agents：智能代理

#### 阶段二：基础组件实践（3-5天）
1. **Models（模型）**
   ```python
   # 安装：pip install langchain langchain-openai
   from langchain_openai import ChatOpenAI
   
   llm = ChatOpenAI(model="gpt-3.5-turbo")
   response = llm.invoke("你好！")
   ```

2. **Prompts（提示词）**
   ```python
   from langchain.prompts import ChatPromptTemplate
   
   template = """你是一个{role}，请用{style}风格回答：
   {question}"""
   
   prompt = ChatPromptTemplate.from_template(template)
   formatted = prompt.format(role="Python导师", 
                           style="简洁明了",
                           quest

#### 2.3.2.1案例1：StrOutputParser

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

# 1. 环境初始化
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

# 2. 初始化模型（无需支持原生结构化输出）
llm = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3
)

# 3. 创建 StrOutputParser
# 核心作用：将 LLM 返回的 AIMessage 对象，统一转为纯字符串（str）
parser = StrOutputParser()

# 4. 链式调用：模型 → 字符串解析
chain = llm | parser
result = chain.invoke("请简要介绍 LangChain 输出解析层的作用")

print("StrOutputParser 解析后的字符串：")
print(result)
print("\n解析结果类型：", type(result))  # str

StrOutputParser 解析后的字符串：
LangChain 的输出解析层（Output Parsers）是**将模型原始输出转换为结构化、可程序化处理数据的关键组件**。它的主要作用如下：

## 核心作用

### 1. **结构化输出**
将LLM的非结构化文本响应（如自由文本）转换为预定义的结构化格式，例如：
- JSON对象
- Python字典/列表
- Pydantic模型实例
- 特定数据类实例

### 2. **数据验证与清洗**
- 验证输出是否符合预期模式
- 自动修复格式错误
- 处理不完整或模糊的响应

### 3. **下游任务集成**
为后续处理提供标准化输入，使LLM输出能够：
- 直接存入数据库
- 作为函数参数传递
- 集成到业务逻辑中
- 用于条件判断和流程控制

## 常用解析器类型

```python
# 示例：使用Pydantic解析器
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str = Field(description="姓名")
    age: int = Field(description="年龄")
    hobbies: list[str] = Field(description="爱好列表")

parser = PydanticOutputParser(pydantic_object=Person)
# 自动生成提示词指令，指导LLM输出正确格式
```

## 工作流程
1. **指令注入**：在提示词中添加格式说明
2. **输出解析**：捕获LLM响应并解析
3. **错误处理**：重试或修复格式错误
4. **类型转换**：将文本转换为目标类型

## 实际价值
- **提高可靠性**：确保输出一致性
- **降低集成成本**：简化后续代码编写
- **增强可维护性**：明确的数据契约
- **支持复杂应用**：实现多步骤链式调用

输出解析层是LangChain构建生产级LLM应用的重要基础设施，它弥合了非结构化文本生成与结构化数据处理之间的鸿沟。

解析结果类

#### 2.3.2.2 案例2： JsonOutputParser

In [15]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# 1. 环境与模型初始化（省略，同方案1）
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"
llm = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3
)

# 2. 创建 JSON 解析器（无需额外配置，默认引导模型输出 JSON）
parser = JsonOutputParser()

# 3. 构建提示模板（无需手动嵌入格式指令，解析器自动关联）
prompt = PromptTemplate(
    template="请介绍1个LangChain开发工具，输出工具名和核心功能。{format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 4. 链式调用（LangChain ≥1.0.0 推荐方式，自动完成提示+调用+解析）
chain = prompt | llm | parser
result = chain.invoke({})  # 无输入参数，传入空字典

print("解析后的JSON（Python字典）：")
print(result)
print("获取单个字段：", result["tool_name"])  # 可直接用于业务逻辑

解析后的JSON（Python字典）：
{'tool_name': 'LangSmith', 'core_function': '提供全链路的LLM应用开发、调试、测试、监控和部署平台，支持追踪和可视化LangChain应用的执行过程，帮助开发者分析性能、诊断问题并优化提示词与工作流。'}
获取单个字段： LangSmith


#### 2.3.2.3 案例3：PydanticOutputParser

In [7]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

# 1. 环境与模型初始化
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

llm = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3
)

# 2. 定义 Pydantic 数据模型
class ToolInfo(BaseModel):
    tool_name: str = Field(description="LangChain开发工具的名称，如 LangSmith")
    function: str = Field(description="工具的核心功能，30字以内")
    difficulty: str = Field(description="学习难度，仅可选：简单 / 中等 / 复杂")

# 3. 创建解析器
parser = PydanticOutputParser(pydantic_object=ToolInfo)

# 4. Prompt + Chain
prompt = PromptTemplate(
    template="请介绍1个 LangChain 开发工具，严格按照要求输出。\n{format_instructions}",
    input_variables=[],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    }
)

chain = prompt | llm | parser
result = chain.invoke({})

print("解析后的结构化数据（Pydantic 模型对象）：")
print(result)

print("字段校验 difficulty：", result.difficulty)

print("转化为字典：", result.model_dump())

解析后的结构化数据（Pydantic 模型对象）：
tool_name='LangSmith' function='用于调试、测试、评估和监控LLM应用的全链路平台' difficulty='中等'
字段校验 difficulty： 中等
转化为字典： {'tool_name': 'LangSmith', 'function': '用于调试、测试、评估和监控LLM应用的全链路平台', 'difficulty': '中等'}


### 2.3.3 BaseOutputParser 核心抽象接口

In [20]:
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# 环境初始化
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"
llm = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3
)

# 自定义解析器
class CustomToolParser(BaseOutputParser):
    def parse(self, text: str) -> dict:
        """将模型输出按 '工具名@核心功能@学习难度' 解析为字典"""
        text = text.strip().replace("\n", "").replace(" ", "")
        parts = text.split("@")
        if len(parts) != 3:
            raise ValueError(f"输出格式错误！需满足「工具名@核心功能@学习难度」，当前输出：{text}")
        return {
            "tool_name": parts[0].strip(),
            "function": parts[1].strip(),
            "difficulty": parts[2].strip()
        }

    def get_format_instructions(self) -> str:
        """生成提示词，引导模型按自定义格式输出"""
        return "请严格按照「工具名@核心功能@学习难度」格式输出，不添加多余内容。示例：LangSmith@全链路调试监控@中等"

# 使用解析器
parser = CustomToolParser()
prompt = PromptTemplate(
    template="请介绍1个LangChain开发工具。{format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
chain = prompt | llm | parser
result = chain.invoke({})

print("自定义解析器解析结果：")
print(result)
print("解析结果类型：", type(result))

自定义解析器解析结果：
{'tool_name': 'LangFlow', 'function': '可视化编排LangChain组件', 'difficulty': '低'}
解析结果类型： <class 'dict'>
